In [5]:
import os
import cv2
from lxml import etree
import xml.etree.cElementTree as ET


# tl: top left; br: bottom right
def writeXml(folder, img, objects, tl, br, saveDir):
    if not os.path.isdir(saveDir):
        os.mkdir(saveDir)

    image = cv2.imread(img.path)
    height, width, depth = image.shape

    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = folder
    ET.SubElement(annotation, 'filename').text = img.name
    ET.SubElement(annotation, 'segmented').text = '0'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str(width)
    ET.SubElement(size, 'height').text = str(height)
    ET.SubElement(size, 'depth').text = str(depth)

    for obj, topl, botr in zip(objects, tl, br):
        ob = ET.SubElement(annotation, 'object')
        ET.SubElement(ob, 'name').text = obj
        ET.SubElement(ob, 'pose').text = 'Unspecified'
        ET.SubElement(ob, 'truncated').text = '0'
        ET.SubElement(ob, 'difficult').text = '0'
        
        bbox = ET.SubElement(ob, 'bndbox')
        ET.SubElement(bbox, 'xmin').text = str(topl[0])
        ET.SubElement(bbox, 'ymin').text = str(topl[1])
        ET.SubElement(bbox, 'xmax').text = str(botr[0])
        ET.SubElement(bbox, 'ymax').text = str(botr[1])
    
    xmlStr = ET.tostring(annotation)
    root = etree.fromstring(xmlStr)
    xmlStr = etree.tostring(root, pretty_print=True)

    save_path = os.path.join(saveDir, img.name.replace('jpg', 'xml'))
    with open(save_path, 'wb') as temp_xml:
        temp_xml.write(xmlStr)
    return xmlStr


if __name__ == '__main__':
    folder = 'pic'
    img = [im for im in os.scandir('pic') if '000001' in im.name][0]
    objects = ['lion']
    tl = [(10, 10)]
    br = [(100, 100)]
    saveDir = 'annotations'
    xmlStr = writeXml(folder, img, objects, tl, br, saveDir)
    print(xmlStr)

b'<annotation>\n  <folder>pic</folder>\n  <filename>0000010.jpg</filename>\n  <segmented>0</segmented>\n  <size>\n    <width>6000</width>\n    <height>4000</height>\n    <depth>3</depth>\n  </size>\n  <object>\n    <name>lion</name>\n    <pose>Unspecified</pose>\n    <truncated>0</truncated>\n    <difficult>0</difficult>\n    <bndbox>\n      <xmin>10</xmin>\n      <ymin>10</ymin>\n      <xmax>100</xmax>\n      <ymax>100</ymax>\n    </bndbox>\n  </object>\n</annotation>\n'
